<a href="https://colab.research.google.com/github/Sunil-1234/Deduplication-using-Record_linkage-in-python/blob/main/Deduplication_using_record_linkage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing required library for record linkage
pip install recordlinkage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 926 kB 30.9 MB/s 
     |████████████████████████████████| 132 kB 66.9 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73989 sha256=f71228139dddef222a98958e4b207601a30a3f368d38d251f6eccd6ced43e77a
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [2]:
import recordlinkage

In [3]:
#import febrl2 dataset from recordlinkage.dataset module
from recordlinkage.datasets import load_febrl2

In [4]:
#loading dataset and specifying “return_links = True” the known duplicate record pairs will be returned.
data, links = load_febrl2(return_links=True)

**Pre-processing the dataset(Data-Standardisation)**

In [5]:
#Lower the each alphabet of dataset(do either uppercase or lowercase)
data=data.astype(str).apply(lambda x:x.str.lower())

In [19]:
#importing nltk library for tokenize and detokenize(important library for NLP)
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer


In [20]:
#defining stopword
name_stopword=['street','st','place','rd','road']
#breaking data present in address_1 into word 
data['address_1_token']=data['address_1'].apply(word_tokenize)
#removing words from address_1_token that are present in stopword list
data['address_1_clean']=data['address_1_token'].apply(lambda x:[word for word in x if word not in name_stopword])
#now, detokenize the address_1_token
data['address_1_clean']=data['address_1_clean'].apply(TreebankWordDetokenizer().detokenize)

In [21]:
#pre-processing postal-code similarly we can preprocess mobile number 
data['postcode']=data['postcode'].str.strip()
data['postcode']=data['postcode'].str.findall('[0-9]+')
data['postcode']=data['postcode'].str.join('')
data['postcode']=data['postcode'].fillna('')


In [22]:
#Removing special character from address_1_clean and address_2
data['address_1_clean'] = data['address_1_clean'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")
data['address_2'] = data['address_2'].str.replace("[\'\".,()*+&\/\-\\\+\!\%:;?]"," ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


**Indexing**

Now our data can be considered as clean set of data , we will need to create pairs of records (also known as candidate links) Pairs records are created and similarities are calculated to determine if the pair of records are considered a match/duplicates. 

There are several indexing technique
1)Full Index(consume more memory as it create every possible pair0
2)Block Index(consume less memory than Full index as we block on particular column)
3)Sorted Neighbourhood(it consumes least memory produces pairs with nearby values)

We may miss real duplicate when we use either block or sorted neighbourhood technique. 
so, in our case we use combination of block index and sorted neighbourhood technique by doing this we may miss very few case and memory consumption will be less

In [37]:
#Full index technique
from recordlinkage.index import Full
Full_Index_Table = Full().index(data)
print(f"table records: {len(data)} records, no of pairs: {len(Full_Index_Table)} pairs")

table records: 5000 records, no of pairs: 12497500 pairs


In [38]:
#Block index technique
from recordlinkage.index import Block
Block_Index_by_State = Block(on="state")
Block_Index_by_State_Pairs = Block_Index_by_State.index(data)
print(f"table records: {len(data)} records, no of pairs: {len(Block_Index_by_State_Pairs)} pairs")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The argument 'on' is deprecated. Use 'left_on=...' and 'right_on=None' to simulate the behaviour of 'on'.
  This is separate from the ipykernel package so we can avoid doing imports until


table records: 5000 records, no of pairs: 2768103 pairs


In [41]:
#Record that are duplicate according to block index technique
Block_Index_by_State_Pairs

MultiIndex([( 'rec-712-dup-0',   'rec-2778-org'),
            (  'rec-63-dup-0',   'rec-2778-org'),
            (  'rec-63-dup-0',  'rec-712-dup-0'),
            (   'rec-112-org',   'rec-2778-org'),
            (   'rec-112-org',  'rec-712-dup-0'),
            (   'rec-112-org',   'rec-63-dup-0'),
            (  'rec-2116-org',   'rec-2778-org'),
            (  'rec-2116-org',  'rec-712-dup-0'),
            (  'rec-2116-org',   'rec-63-dup-0'),
            (  'rec-2116-org',    'rec-112-org'),
            ...
            (   'rec-330-org',    'rec-502-org'),
            (   'rec-330-org', 'rec-2467-dup-2'),
            (   'rec-330-org',   'rec-3107-org'),
            (   'rec-330-org',     'rec-93-org'),
            (   'rec-330-org',  'rec-419-dup-0'),
            (   'rec-330-org',   'rec-1119-org'),
            (   'rec-330-org',   'rec-2333-org'),
            (   'rec-330-org', 'rec-3703-dup-2'),
            (   'rec-330-org',   'rec-3909-org'),
            ('rec-3231-dup-0',  'r

In [39]:
from recordlinkage.index import SortedNeighbourhood
Neighbour_Index_by_Name = SortedNeighbourhood(on="surname", window = 5)
Neighbour_Index_by_Name_Pairs = Neighbour_Index_by_Name.index(data)
print(f"table records: {len(data)} records, no of pairs: {len(Neighbour_Index_by_Name_Pairs)} pairs")

table records: 5000 records, no of pairs: 75034 pairs


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The argument 'on' is deprecated. Use 'left_on=...' and 'right_on=None' to simulate the behaviour of 'on'.
  


In [40]:
#Record that are duplicate according to Sorted Neighbourhood technique
Neighbour_Index_by_Name_Pairs

MultiIndex([('rec-712-dup-2',    'rec-215-org'),
            (  'rec-712-org',    'rec-215-org'),
            ( 'rec-2084-org',    'rec-215-org'),
            ( 'rec-1135-org', 'rec-3409-dup-0'),
            ( 'rec-2108-org', 'rec-3409-dup-0'),
            ( 'rec-2108-org',   'rec-3409-org'),
            (   'rec-71-org', 'rec-3409-dup-0'),
            (   'rec-71-org',   'rec-3409-org'),
            ( 'rec-3672-org', 'rec-3409-dup-0'),
            ( 'rec-3672-org',   'rec-3409-org'),
            ...
            (  'rec-322-org',    'rec-222-org'),
            (  'rec-322-org',   'rec-2049-org'),
            (  'rec-322-org',   'rec-2975-org'),
            (  'rec-322-org',    'rec-498-org'),
            (  'rec-322-org',    'rec-396-org'),
            ( 'rec-3433-org',   'rec-2065-org'),
            ( 'rec-3812-org',   'rec-1485-org'),
            (  'rec-303-org',   'rec-2719-org'),
            (  'rec-303-org',   'rec-3230-org'),
            (  'rec-303-org',    'rec-162-org')],
   

In [26]:
#Combine the block index technique and Sorted Neighbourhood technique Together
All_Index_Pairs = Block_Index_by_State_Pairs.append(Neighbour_Index_by_Name_Pairs)
#Remove duplicate Pairs that is common in both technique
All_Index_Pairs = All_Index_Pairs.drop_duplicates(keep='first')

In [42]:
#Record that are duplicate according to combination of block indexing and sorted neighbourhood 
All_Index_Pairs

MultiIndex([('rec-712-dup-0',  'rec-2778-org'),
            ( 'rec-63-dup-0',  'rec-2778-org'),
            ( 'rec-63-dup-0', 'rec-712-dup-0'),
            (  'rec-112-org',  'rec-2778-org'),
            (  'rec-112-org', 'rec-712-dup-0'),
            (  'rec-112-org',  'rec-63-dup-0'),
            ( 'rec-2116-org',  'rec-2778-org'),
            ( 'rec-2116-org', 'rec-712-dup-0'),
            ( 'rec-2116-org',  'rec-63-dup-0'),
            ( 'rec-2116-org',   'rec-112-org'),
            ...
            ( 'rec-3587-org',  'rec-1541-org'),
            ( 'rec-3005-org',  'rec-3647-org'),
            (  'rec-322-org',   'rec-232-org'),
            (  'rec-322-org',  'rec-1304-org'),
            (  'rec-322-org',  'rec-1523-org'),
            (  'rec-322-org',  'rec-2049-org'),
            (  'rec-322-org',  'rec-2975-org'),
            (  'rec-322-org',   'rec-396-org'),
            ( 'rec-3812-org',  'rec-1485-org'),
            (  'rec-303-org',  'rec-2719-org')],
           names=['rec_

**Comparison & Similarity**

Now, our record pair is generated, we would like to calculate similarity score of these pair.
There are various algorithm for calculating similarity Score
1)Jarowinkler
2)Levenshtein
3)Longest Common Substring (LCS)
4)Jaccard

In [46]:
# initialise class 
compare = recordlinkage.Compare()

# initialise similarity measurement algorithms
compare.string('given_name','given_name', method='jarowinkler', label = 'given_name_score')# we can also pass threshold as another parameter
compare.string('surname','surname', method='jarowinkler', label = 'surname_score')
compare.string('street_number','street_number', method='levenshtein', label = 'street_number_score')
compare.string('address_1_clean','address_1_clean', method='jarowinkler', label = 'address_1_score')
compare.string('address_2','address_2', method='jarowinkler', label = 'address_2_score')
compare.string('suburb','suburb', method='jarowinkler', label = 'suburb_score')
compare.string('postcode','postcode', method='levenshtein', label = 'postcode_score')
compare.string('state','state', method='jarowinkler', label = 'state_score')
compare.string('date_of_birth','date_of_birth', method='levenshtein', label = 'date_of_birth_score')
compare.string('soc_sec_id','soc_sec_id', method='levenshtein', label = 'soc_sec_id_score')

# the method .compute() returns the DataFrame with the feature vectors.
comparison_vectors = compare.compute(All_Index_Pairs,data) #computing compare record for all possible pair that are present in All_Index_Pair

In [47]:
comparison_vectors

given_name_score  surname_score  \
rec_id_1      rec_id_2                                         
rec-712-dup-0 rec-2778-org           0.466667       0.455556   
rec-63-dup-0  rec-2778-org           0.455556       0.000000   
              rec-712-dup-0          0.000000       0.000000   
rec-112-org   rec-2778-org           0.588889       0.633333   
              rec-712-dup-0          0.577778       0.000000   
...                                       ...            ...   
rec-322-org   rec-2049-org           0.444444       0.822222   
              rec-2975-org           0.455556       0.822222   
              rec-396-org            0.444444       0.822222   
rec-3812-org  rec-1485-org           0.428571       0.611111   
rec-303-org   rec-2719-org           0.436508       0.666667   

                             street_number_score  address_1_score  \
rec_id_1      rec_id_2                                              
rec-712-dup-0 rec-2778-org              0.000000         0.422222   
rec-63-dup-0  rec-2778-org              0.000000         0.455556   
              rec-712-dup-0             0.500000         0.000000   
rec-112-org   rec-2778-org              0.000000         0.444444   
              rec-712-dup-0             0.000000         0.588889   
...                                          ...              ...   
rec-322-org   rec-2049-org              0.000000         0.466667   
              rec-2975-org              0.000000         0.441667   
              rec-396-org               0.333333         0.561905   
rec-3812-org  rec-1485-org              0.000000         0.586193   
rec-303-org   rec-2719-org              0.333333         0.558333   

                             address_2_score  suburb_score  postcode_score  \
rec_id_1      rec_id_2                                                       
rec-712-dup-0 rec-2778-org          0.657143      0.514550            0.00   
rec-63-dup-0  rec-2778-org          0.547222      0.405556            0.25   
              rec-712-dup-0         0.401786      0.457407            0.50   
rec-112-org   rec-2778-org          0.665476      0.388889            0.25   
              rec-712-dup-0         0.535714      0.514550            0.25   
...                                      ...           ...             ...   
rec-322-org   rec-2049-org          0.000000      0.000000            0.00   
              rec-2975-org          0.455988      0.464286            0.25   
              rec-396-org           0.436508      0.511905            0.00   
rec-3812-org  rec-1485-org          0.501166      0.344444            0.00   
rec-303-org   rec-2719-org          0.492063      0.472222            0.00   

                             state_score  date_of_birth_score  \
rec_id_1      rec_id_2                                          
rec-712-dup-0 rec-2778-org           1.0                0.500   
rec-63-dup-0  rec-2778-org           1.0                0.500   
              rec-712-dup-0          1.0                0.500   
rec-112-org   rec-2778-org           1.0                0.250   
              rec-712-dup-0          1.0                0.375   
...                                  ...                  ...   
rec-322-org   rec-2049-org           0.0                0.250   
              rec-2975-org           0.0                0.500   
              rec-396-org            0.0                0.500   
rec-3812-org  rec-1485-org           0.0                0.625   
rec-303-org   rec-2719-org           0.0                0.500   

                             soc_sec_id_score  
rec_id_1      rec_id_2                         
rec-712-dup-0 rec-2778-org           0.000000  
rec-63-dup-0  rec-2778-org           0.000000  
              rec-712-dup-0          0.142857  
rec-112-org   rec-2778-org           0.000000  
              rec-712-dup-0          0.285714  
...                                       ...  
rec-322-org   rec-2049-org           0.000000  
     

**Supervised Learning (Classification)**

we will train a model to classify duplicates and non-duplicates based on the data set provided. But before we can train the model, we will need to have a “label” column (Target Variable) in our data set for the model to know which are duplicates and which are not.

In [50]:
# computing compare score that are most likely to be duplicates
duplicate_pairs_vectors = compare.compute(links,data)

In [52]:
duplicate_pairs_vectors

given_name_score  surname_score  \
rec-712-dup-1  rec-712-dup-0           1.000000       0.933333   
rec-712-dup-2  rec-712-dup-0           0.961111       1.000000   
               rec-712-dup-1           0.961111       0.933333   
rec-712-org    rec-712-dup-0           1.000000       1.000000   
               rec-712-dup-1           1.000000       0.933333   
...                                         ...            ...   
rec-1035-dup-0 rec-1035-org            0.444444       1.000000   
rec-2915-org   rec-2915-dup-0          0.500000       1.000000   
rec-678-dup-0  rec-678-org             1.000000       0.981818   
rec-1250-org   rec-1250-dup-0          0.975000       1.000000   
rec-526-dup-0  rec-526-org             1.000000       1.000000   

                               street_number_score  address_1_score  \
rec-712-dup-1  rec-712-dup-0                   1.0         1.000000   
rec-712-dup-2  rec-712-dup-0                   1.0         1.000000   
               rec-712-dup-1                   1.0         1.000000   
rec-712-org    rec-712-dup-0                   1.0         1.000000   
               rec-712-dup-1                   1.0         1.000000   
...                                            ...              ...   
rec-1035-dup-0 rec-1035-org                    1.0         1.000000   
rec-2915-org   rec-2915-dup-0                  1.0         1.000000   
rec-678-dup-0  rec-678-org                     0.0         0.985714   
rec-1250-org   rec-1250-dup-0                  0.5         0.987500   
rec-526-dup-0  rec-526-org                     1.0         1.000000   

                               address_2_score  suburb_score  postcode_score  \
rec-712-dup-1  rec-712-dup-0          0.975000      0.977778             1.0   
rec-712-dup-2  rec-712-dup-0          0.000000      0.527778             1.0   
               rec-712-dup-1          0.000000      0.500926             1.0   
rec-712-org    rec-712-dup-0          0.975000      1.000000             1.0   
               rec-712-dup-1          1.000000      0.977778             1.0   
...                                        ...           ...             ...   
rec-1035-dup-0 rec-1035-org           1.000000      0.985714             1.0   
rec-2915-org   rec-2915-dup-0         1.000000      0.450000             1.0   
rec-678-dup-0  rec-678-org            1.000000      1.000000             1.0   
rec-1250-org   rec-1250-dup-0         1.000000      1.000000             1.0   
rec-526-dup-0  rec-526-org            0.989474      1.000000             1.0   

                               state_score  date_of_birth_score  \
rec-712-dup-1  rec-712-dup-0           1.0                1.000   
rec-712-dup-2  rec-712-dup-0           1.0                1.000   
               rec-712-dup-1           1.0                1.000   
rec-712-org    rec-712-dup-0           1.0                1.000   
               rec-712-dup-1           1.0                1.000   
...                                    ...                  ...   
rec-1035-dup-0 rec-1035-org            1.0                1.000   
rec-2915-org   rec-2915-dup-0          1.0                1.000   
rec-678-dup-0  rec-678-org             1.0                1.000   
rec-1250-org   rec-1250-dup-0          1.0                0.375   
rec-526-dup-0  rec-526-org             1.0                1.000   

                               soc_sec_id_score  
rec-712-dup-1  rec-712-dup-0                1.0  
rec-712-dup-2  rec-712-dup-0                1.0  
               rec-712-dup-1                1.0  
rec-712-org    rec-712-dup-0                1.0  
               rec-712-dup-1                1.0  
...                                         ...  
rec-1035-dup-0 rec-1035-org                 1.0  
rec-2915-org   rec-2915-dup-0               1.0  
rec-678-dup-0  rec-678-org                  1.0  
rec-1250-org   rec-1250-dup-0               1.0  
rec-526-dup-0  rec-526-org                  1.0  

[1934 rows x 10 co

In [48]:

import pandas as pd
import numpy as np

The following steps are some ETL processes to create the column “Label” on our data set whereby if the pairing is found in the data set “duplicate_pairs” then is label as “1” else “0”

In [30]:
duplicate_pairs = duplicate_pairs_vectors.reset_index()
duplicate_pairs_1 = duplicate_pairs["level_0"]+','+duplicate_pairs["level_1"]
duplicate_pairs_2 = duplicate_pairs["level_1"]+','+duplicate_pairs["level_0"]
final_duplicate_pairs = pd.DataFrame(duplicate_pairs_1.append(duplicate_pairs_2))
comparison_pairs = comparison_vectors.reset_index()
comparison_pairs['join_keys'] = comparison_pairs["rec_id_1"]+','+comparison_pairs["rec_id_2"]
# 1 represent Duplicates, 0 represent non duplicates
comparison_pairs['Label'] = np.where(comparison_pairs["join_keys"].isin(final_duplicate_pairs[0]),'1','0')
comparison_pairs.groupby(['Label'])['join_keys'].count()

Label
0    2824073
1       1901
Name: join_keys, dtype: int64

Now we have a set of labeled data, we can begin training a supervised learning model to classify the records as “duplicate” or “not duplicate”.

In [31]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#Index Composite Variables(we use ['join_keys','rec_id_1','rec_id_2'] as a index for new data set 'Model_Data_Set' )
Model_Data_Set = comparison_pairs.set_index(['join_keys','rec_id_1','rec_id_2'])

#Split Data Into Label and Features
y= Model_Data_Set['Label']
x= Model_Data_Set.drop(['Label'],axis=1)

#Sppliting dataset into Training & Test Set 
seed = 1
test_size = 0.4 # using 60 % of our data for training our model
x_train, x_test, y_train, y_test = model_selection.train_test_split(x,y,test_size=test_size, random_state=seed, stratify=y)

pd.Series(y_test).value_counts()

0    1129630
1        760
Name: Label, dtype: int64

Now, we train the data and will predict the output

In [33]:
#Apply XGB Model 

model= xgb.XGBClassifier(learning_rate = 0.05, n_estimators=300, max_depth=5)
model.fit(x_train, y_train)
print(model)

y_pred = pd.DataFrame(model.predict(x_test))
predictions = y_pred
predictions['predict'] = y_pred

dfcombine = pd.merge(x_test.reset_index(),predictions[['predict']],how='left',left_index= True, right_index = True)
dfcombine


XGBClassifier(learning_rate=0.05, max_depth=5, n_estimators=300)


,join_keys,rec_id_1,rec_id_2,given_name_score,surname_score,street_number_score,address_1_score,address_2_score,suburb_score,postcode_score,state_score,date_of_birth_score,soc_sec_id_score,predict
0,"rec-645-org,rec-1753-org",rec-645-org,rec-1753-org,0.611111,0.430303,0.000000,0.477778,0.436508,0.472222,0.25,1.0,0.625,0.000000,0
1,"rec-1900-dup-2,rec-455-dup-3",rec-1900-dup-2,rec-455-dup-3,0.555556,0.464286,0.000000,0.351282,0.474747,0.466667,0.00,1.0,0.000,0.285714,0
2,"rec-3122-org,rec-3404-org",rec-3122-org,rec-3404-org,0.527778,0.000000,0.500000,0.493651,0.313187,0.433333,0.00,1.0,0.500,0.000000,0
3,"rec-589-dup-2,rec-1524-org",rec-589-dup-2,rec-1524-org,0.676190,0.464286,0.000000,0.561905,0.345238,0.473016,0.00,1.0,0.500,0.142857,0
4,"rec-805-org,rec-3371-org",rec-805-org,rec-3371-org,0.502646,0.464286,0.666667,0.374074,0.542857,0.607143,0.25,1.0,0.500,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130385,"rec-1921-org,rec-3517-org",rec-1921-org,rec-3517-org,0.441667,0.447619,0.000000,0.412037,0.567460,0.395833,0.50,1.0,0.375,0.285714,0
1130386,"rec-3112-org,rec-1332-org",rec-3112-org,rec-1332-org,0.583333,0.539683,0.500000,0.561905,0.418651,0.417989,0.25,1.0,0.500,0.142857,0
1130387,"rec-3122-org,rec-1837-org",rec-3122-org,rec-1837-org,0.777778,0.527778,0.500000,0.490079,0.000000,0.374074,0.25,1.0,0.500,0.000000,0
1130388,"rec-3115-org,rec-1989-org",rec-3115-org,rec-1989-org,0.550000,0.527778,1.000000,0.412037,0.433333,0.614815,0.25,1.0,0.250,0.285714,0


In [34]:
dfcombine.loc[dfcombine['predict']=='1']

,join_keys,rec_id_1,rec_id_2,given_name_score,surname_score,street_number_score,address_1_score,address_2_score,suburb_score,postcode_score,state_score,date_of_birth_score,soc_sec_id_score,predict
3189,"rec-3563-dup-1,rec-3563-org",rec-3563-dup-1,rec-3563-org,1.000000,1.000000,1.0,0.885714,0.933333,1.000000,1.0,1.0,1.00,1.000000,1
8268,"rec-3839-dup-1,rec-3839-org",rec-3839-dup-1,rec-3839-org,1.000000,0.971429,1.0,1.000000,0.933333,1.000000,1.0,1.0,1.00,1.000000,1
8426,"rec-1943-dup-1,rec-1943-org",rec-1943-dup-1,rec-1943-org,1.000000,0.921429,1.0,1.000000,1.000000,0.504545,1.0,1.0,1.00,1.000000,1
12481,"rec-88-org,rec-88-dup-0",rec-88-org,rec-88-dup-0,1.000000,1.000000,1.0,0.947381,1.000000,1.000000,1.0,1.0,0.75,1.000000,1
16462,"rec-1213-dup-4,rec-1213-dup-1",rec-1213-dup-4,rec-1213-dup-1,0.893333,1.000000,1.0,0.388889,0.592593,1.000000,1.0,1.0,1.00,0.857143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1124717,"rec-1585-dup-1,rec-1585-dup-3",rec-1585-dup-1,rec-1585-dup-3,0.574074,0.975000,1.0,0.430556,0.301282,0.948333,1.0,1.0,1.00,1.000000,1
1124813,"rec-1477-dup-0,rec-1477-org",rec-1477-dup-0,rec-1477-org,1.000000,1.000000,1.0,1.000000,0.486111,0.951471,1.0,1.0,1.00,1.000000,1
1125852,"rec-3968-dup-0,rec-3968-dup-1",rec-3968-dup-0,rec-3968-dup-1,0.966667,1.000000,1.0,1.000000,0.916667,1.000000,0.5,0.0,1.00,1.000000,1
1126876,"rec-1010-dup-1,rec-1010-dup-0",rec-1010-dup-1,rec-1010-dup-0,0.518519,0.511905,1.0,0.423810,0.477778,1.000000,1.0,1.0,1.00,1.000000,1


In [36]:
dfcombine[dfcombine['join_keys']=='rec-3563-dup-1,rec-3563-org']

,join_keys,rec_id_1,rec_id_2,given_name_score,surname_score,street_number_score,address_1_score,address_2_score,suburb_score,postcode_score,state_score,date_of_birth_score,soc_sec_id_score,predict
3189,"rec-3563-dup-1,rec-3563-org",rec-3563-dup-1,rec-3563-org,1.0,1.0,1.0,0.885714,0.933333,1.0,1.0,1.0,1.0,1.0,1
